In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
import numpy as np
from sklearn.metrics import r2_score


from numpy.random import default_rng
RANDOM_STATE = 2 # random state for default_rng
rng = default_rng(RANDOM_STATE)


import random
#import altair as alt
from tqdm import tqdm
import datetime

%load_ext line_profiler



# CHECKLIST for Kaggle variant
# Use FAST_PASS True on first pass
# USE_ALL_STOCK_IDS False to check then True
# USE_TEST_LOCAL_6_ITEMS must be False else we override the local test data
# TRAIN_TEST False
# Check on Kaggle that "internet" is disabled
# First run with "USE_ALL_STOCK_IDS=False", flip to True, Save Version, it'll take 30 mins to run

# CHECKLIST for home variant
# USE_ALL_STOCK_IDS False for fast dev, True for proper testing
# USE_TEST_LOCAL_6_ITEMS False for fast dev, True for proper testing
# NBR_FOR_SUBSET_OF_STOCK_IDS 4 for quick testing

t1_notebook_start = datetime.datetime.utcnow()

if os.environ.get('USER') == 'ian':
    ENV_HOME = True
    import ipython_memory_usage
    %ipython_memory_usage_start
    USE_ALL_STOCK_IDS = True
    NBR_FOR_SUBSET_OF_STOCK_IDS = 4
    TRAIN_TEST = True
    USE_TEST_LOCAL_6_ITEMS = False # robust local testing at home
    MEMORY_LOCATION = 'joblib_cache'

if USE_ALL_STOCK_IDS:
    NBR_FOR_SUBSET_OF_STOCK_IDS=None
print(f'ENV_HOME: {ENV_HOME}, TRAIN_TEST {TRAIN_TEST}, USE_ALL_STOCK_IDS {USE_ALL_STOCK_IDS}, USE_TEST_LOCAL_6_ITEMS {USE_TEST_LOCAL_6_ITEMS}')
print(f'NBR_FOR_SUBSET_OF_STOCK_IDS: {NBR_FOR_SUBSET_OF_STOCK_IDS}')

ENV_HOME: True, TRAIN_TEST False, USE_ALL_STOCK_IDS False, USE_TEST_LOCAL_6_ITEMS False
NBR_FOR_SUBSET_OF_STOCK_IDS: 4
In [1] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 159.54 MiB


In [2]:
# OR PASTE IN UTILITY CODE HERE FOR KAGGLE
from utility import make_unique_time_ids, get_training_stock_ids, rmspe_score
from utility import ROOT, TEST_CSV, TRAIN_CSV

Utility says ROOT is /home/ian/data/kaggle/optiver_volatility/
In [2] used 0.0859 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 159.63 MiB


## Load train set

In [3]:
df_train_all = pd.read_csv(TRAIN_CSV)
df_train_all = df_train_all.set_index(['stock_id', 'time_id'])
print(df_train_all.shape)
#rows_for_stock_id_0 = df_train_all.query('stock_id == 0').shape[0]
#rows_for_stock_id_0

(428932, 1)
In [3] used 20.9805 MiB RAM in 0.29s, peaked 7.46 MiB above current, total RAM usage 180.61 MiB


In [5]:
def load_book(ROOT, filename, stock_id):
    df_book_train_stock_X = pd.read_parquet(os.path.join(ROOT, f"{filename}/stock_id={stock_id}"))
    df_book_train_stock_X["stock_id"] = stock_id
    df_book_train_stock_X = df_book_train_stock_X.set_index(['stock_id', 'time_id'])
    return df_book_train_stock_X

df_book_train_stock_X = load_book(ROOT, 'book_train.parquet', 0)

In [5] used 175.0078 MiB RAM in 0.31s, peaked 0.00 MiB above current, total RAM usage 356.34 MiB


In [6]:
def show_details(df):
    try:
        nbr_index_levels = len(df.index.levels)
    except AttributeError:
        nbr_index_levels = 1
    nbr_nulls = df.isnull().sum().sum()
    #nulls_msg = "Has no nulls"
    #if nbr_nulls==0:
    nulls_msg = f"{nbr_nulls} nulls"
    is_view_msg = f'is_view {df_train_all._data.is_view}'
    is_single_block_msg = f'is_single_block {df_train_all._data.is_single_block}'
    is_consolidated_msg = f'is_consolidated {df_train_all._data.is_consolidated()}'    
    print(f'[{nbr_index_levels}c] {df.shape[0]:,}x{df.shape[1]:,}, {nulls_msg}, {is_view_msg}, {is_single_block_msg}, {is_consolidated_msg}')

show_details(df_train_all)

[2c] 428,932x1, 0 nulls, is_view True, is_single_block True, is_consolidated True
In [6] used -14.0000 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 342.34 MiB


In [8]:
# make feature columns
def make_features_stats(df_book, agg_type, cols):
    features_var1 = df_book.groupby(['stock_id', 'time_id'])[cols].agg(agg_type)
    #print(type(features_var1))
    if isinstance(features_var1, pd.Series):
        # .size yields a series not a df
        #features_var1.name = str(agg_type)
        features_var1 = pd.DataFrame(features_var1, columns=[agg_type])
        #pass
    else:
        features_var1_col_names = [f"{col}_{agg_type}" for col in cols]
        features_var1.columns = features_var1_col_names
    return features_var1

if True: # lightweight tests
    df_book_train_stock_XX = pd.read_parquet(os.path.join(ROOT, f"book_train.parquet/stock_id=0"))
    df_book_train_stock_XX["stock_id"] = 0
    df_book_train_stock_XX = df_book_train_stock_XX.set_index(['stock_id', 'time_id'])
    display(make_features_stats(df_book_train_stock_XX, 'nunique', ['ask_size1']).head())

ask_size1_nunique
stock_id time_id                   
0        5                       67
         11                      26
         16                      22
         31                      30
         62                      54

In [8] used 142.0273 MiB RAM in 0.47s, peaked 40.01 MiB above current, total RAM usage 399.69 MiB


In [9]:
%timeit make_features_stats(df_book_train_stock_XX, 'nunique', ['ask_size1'])

172 ms ± 6.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
In [9] used -102.0781 MiB RAM in 1.52s, peaked 70.17 MiB above current, total RAM usage 297.61 MiB


In [10]:
%lprun -f make_features_stats make_features_stats(df_book_train_stock_XX, 'nunique', ['ask_size1'])

In [10] used 0.0156 MiB RAM in 0.32s, peaked 44.01 MiB above current, total RAM usage 297.63 MiB


Timer unit: 1e-06 s

Total time: 0.216714 s
File: /tmp/ipykernel_66873/3623217067.py
Function: make_features_stats at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def make_features_stats(df_book, agg_type, cols):
     3         1     216386.0 216386.0     99.8      features_var1 = df_book.groupby(['stock_id', 'time_id'])[cols].agg(agg_type)
     4                                               #print(type(features_var1))
     5         1          5.0      5.0      0.0      if isinstance(features_var1, pd.Series):
     6                                                   # .size yields a series not a df
     7                                                   #features_var1.name = str(agg_type)
     8                                                   features_var1 = pd.DataFrame(features_var1, columns=[agg_type])
     9                                                   #pass
    10                                  

In [73]:
def _realized_volatility_weighted_sub(ser, weights):
    ser_weighted = ser * weights
    return np.sqrt(np.sum(ser_weighted**2))

def realized_volatility_weighted(ser, weights_type):
    """Weighted volatility"""
    # as a numpy array
    # we drop from 12us to 3us by adding @njit to the _sub function
    # we can't make _sub a closure, it loses all compilation benefits
    # and we can't add njit(cache=True) in Jupyter as it can't
    # find a cache location    
    # as a Series we have 5us and 15us w/wo @njit respectively
    if isinstance(ser, pd.Series):
        ser = ser.to_numpy()
    nbr_items = ser.shape[0]
    if weights_type == 'uniform':
        weights = np.ones(nbr_items)
    elif weights_type == 'linear':
        weights = np.linspace(0.1, 1, nbr_items) # linear increasing weight
    elif weights_type == 'half0half1':
        half_way = int(ser.shape[0] / 2)
        weights = np.concatenate((np.zeros(half_way), np.ones(ser.shape[0] - half_way))) # 0s then 1s weight
    elif weights_type == 'geom':
        weights = np.geomspace(0.01, 1, nbr_items) # geometric increase
    #assert isinstance(weights_type, str) == False, f"Must not be a string like '{weights}' at this point"
    return _realized_volatility_weighted_sub(ser, weights)

#from joblib import Memory
#memory = Memory("joblib_cache", verbose=0)
#@memory.cache

from functools import lru_cache

@lru_cache
def get_weights(nbr_items, weights_type):
    if weights_type == 'uniform':
        weights = np.ones(nbr_items)
    elif weights_type == 'linear':
        weights = np.linspace(0.1, 1, nbr_items) # linear increasing weight
    elif weights_type == 'half0half1':
        half_way = int(ser.shape[0] / 2)
        weights = np.concatenate((np.zeros(half_way), np.ones(ser.shape[0] - half_way))) # 0s then 1s weight
    elif weights_type == 'geom':
        weights = np.geomspace(0.01, 1, nbr_items) # geometric increase
    return weights

def realized_volatility_weighted_v2(ser, weights_type):
    """Weighted volatility"""
    # as a numpy array
    # we drop from 12us to 3us by adding @njit to the _sub function
    # we can't make _sub a closure, it loses all compilation benefits
    # and we can't add njit(cache=True) in Jupyter as it can't
    # find a cache location    
    # as a Series we have 5us and 15us w/wo @njit respectively
    if isinstance(ser, pd.Series):
        ser = ser.to_numpy()
    nbr_items = ser.shape[0]
    weights = get_weights(nbr_items, weights_type)
    #assert isinstance(weights_type, str) == False, f"Must not be a string like '{weights}' at this point"
    return _realized_volatility_weighted_sub(ser, weights)

series_log_return = pd.Series(np.linspace(0, 10, 600))
print(realized_volatility_weighted(series_log_return, weights_type="uniform"))
assert realized_volatility_weighted(series_log_return, weights_type="uniform") == realized_volatility_weighted_v2(series_log_return, weights_type="uniform")
    #%timeit realized_volatility_weighted(series_log_return, weights_type="uniform")

141.48036786359452
In [73] used 0.2539 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 634.11 MiB


In [68]:
%timeit realized_volatility_weighted(series_log_return, weights_type="uniform")

18.5 µs ± 589 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
In [68] used 0.0000 MiB RAM in 1.66s, peaked 0.00 MiB above current, total RAM usage 633.83 MiB


In [75]:
%timeit realized_volatility_weighted_v2(series_log_return, weights_type="uniform")

13.9 µs ± 207 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
In [75] used 0.0000 MiB RAM in 11.42s, peaked 0.00 MiB above current, total RAM usage 634.11 MiB


In [13]:
%lprun -f realized_volatility_weighted -f _realized_volatility_weighted_sub realized_volatility_weighted(series_log_return, weights_type="uniform")

In [13] used 0.0312 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 297.68 MiB


Timer unit: 1e-06 s

Total time: 7.9e-05 s
File: /tmp/ipykernel_66873/3070439160.py
Function: _realized_volatility_weighted_sub at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def _realized_volatility_weighted_sub(ser, weights):
     2         1         24.0     24.0     30.4      ser_weighted = ser * weights
     3         1         55.0     55.0     69.6      return np.sqrt(np.sum(ser_weighted**2))

Total time: 0.00017 s
File: /tmp/ipykernel_66873/3070439160.py
Function: realized_volatility_weighted at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def realized_volatility_weighted(ser, weights_type):
     6                                               """Weighted volatility"""
     7                                               # as a numpy array
     8                                               # we drop from 12us to 3us by adding @

In [14]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

#def realized_volatility(series_log_return):
#    return np.sqrt(np.sum(series_log_return**2))

def make_wap(df_book_data, num=1, wap_colname="wap"):
    """Modifies df_book_data"""
    assert num==1 or num==2
    wap_numerator = (df_book_data[f'bid_price{num}'] * df_book_data[f'ask_size{num}'] +
                                     df_book_data[f'ask_price{num}'] * df_book_data[f'bid_size{num}'])
    wap_denominator = df_book_data[f'bid_size{num}'] + df_book_data[f'ask_size{num}']
    df_book_data[wap_colname] = wap_numerator / wap_denominator

#@memory.cache
def make_realized_volatility(df_book_data, log_return_name='log_return', wap_colname='wap', weights=None):
    """Consume wap column"""
    df_book_data[log_return_name] = df_book_data.groupby(['stock_id', 'time_id'])[wap_colname].apply(log_return)
    df_book_data = df_book_data[~df_book_data[log_return_name].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['stock_id', 'time_id'])[log_return_name].agg(realized_volatility_weighted, weights))
    return df_realized_vol_per_stock

make_wap(df_book_train_stock_X, 2) # adds 'wap' column
df_realized_vol_per_stockX = make_realized_volatility(df_book_train_stock_X, log_return_name="log_return2", weights='linear')
display(df_realized_vol_per_stockX.head(2))

log_return2
stock_id time_id             
0        5           0.004500
         11          0.001749

In [14] used 89.9141 MiB RAM in 2.79s, peaked 39.50 MiB above current, total RAM usage 387.59 MiB


In [15]:
def make_volatility_ask_bid_diff(df_book_data, col='ask1_bid1_diff', weights='uniform'):
    new_name = col + "_log_return_" + weights
    df_book_data[new_name] = df_book_data.groupby(['stock_id', 'time_id'])[col].apply(log_return)
    df_book_data = df_book_data[~df_book_data[new_name].isnull()]
    # makes a new dataframe
    df_realized_vol_ask_bid_diff_per_stock =  pd.DataFrame(df_book_data.groupby(['stock_id', 'time_id'])[new_name].agg(realized_volatility_weighted, weights))
    return df_realized_vol_ask_bid_diff_per_stock

df_book_train_stock_X['ask1_bid1_diff'] = (df_book_train_stock_X['ask_price1'] / df_book_train_stock_X['bid_price1']) - 1
df_realized_vol_ask_bid_diff_per_stock = make_volatility_ask_bid_diff(df_book_train_stock_X, weights='linear')
display(df_realized_vol_ask_bid_diff_per_stock.head(2))

ask1_bid1_diff_log_return_linear
stock_id time_id                                  
0        5                                2.032456
         11                               1.646661

In [15] used 9.5977 MiB RAM in 2.50s, peaked 55.87 MiB above current, total RAM usage 397.19 MiB


In [18]:
%lprof -f make_volatility_ask_bid_diff make_volatility_ask_bid_diff(df_book_train_stock_X, weights='linear')

In [18] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 414.73 MiB


UsageError: Line magic function `%lprof` not found.


In [16]:
def make_wap(df_book_data, num=1, wap_colname="wap"):
    """Modifies df_book_data"""
    assert num==1 or num==2
    wap_numerator = (df_book_data[f'bid_price{num}'] * df_book_data[f'ask_size{num}'] +
                                     df_book_data[f'ask_price{num}'] * df_book_data[f'bid_size{num}'])
    wap_denominator = df_book_data[f'bid_size{num}'] + df_book_data[f'ask_size{num}']
    df_book_data[wap_colname] = wap_numerator / wap_denominator

def make_realized_volatility(df_book_data, log_return_name='log_return', wap_colname='wap', weights=None):
    """Consume wap column"""
    df_book_data[log_return_name] = df_book_data.groupby(['stock_id', 'time_id'])[wap_colname].apply(log_return)
    df_book_data = df_book_data[~df_book_data[log_return_name].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['stock_id', 'time_id'])[log_return_name].agg(realized_volatility_weighted, weights))
    return df_realized_vol_per_stock

make_wap(df_book_train_stock_X, 2) # adds 'wap' column
df_realized_vol_per_stockX = make_realized_volatility(df_book_train_stock_X, log_return_name="log_return2", weights='linear')
display(df_realized_vol_per_stockX.head(2))

log_return2
stock_id time_id             
0        5           0.004500
         11          0.001749

In [16] used -15.9570 MiB RAM in 2.48s, peaked 71.86 MiB above current, total RAM usage 381.23 MiB


In [22]:
%lprun -f make_realized_volatility -f realized_volatility_weighted -f _realized_volatility_weighted_sub -f log_return make_realized_volatility(df_book_train_stock_X, log_return_name="log_return2", weights='linear')

In [22] used -0.3008 MiB RAM in 5.23s, peaked 35.43 MiB above current, total RAM usage 428.05 MiB


Timer unit: 1e-06 s

Total time: 0.090863 s
File: /tmp/ipykernel_66873/3070439160.py
Function: _realized_volatility_weighted_sub at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def _realized_volatility_weighted_sub(ser, weights):
     2      3830       9869.0      2.6     10.9      ser_weighted = ser * weights
     3      3830      80994.0     21.1     89.1      return np.sqrt(np.sum(ser_weighted**2))

Total time: 0.456658 s
File: /tmp/ipykernel_66873/3070439160.py
Function: realized_volatility_weighted at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def realized_volatility_weighted(ser, weights_type):
     6                                               """Weighted volatility"""
     7                                               # as a numpy array
     8                                               # we drop from 12us to 3us by adding

In [26]:
ser = df_book_train_stock_X.query('stock_id ==0 and time_id == 5')['wap']
log_return(ser)

stock_id  time_id
0         5               NaN
          5          0.000000
          5          0.000001
          5          0.000000
          5          0.000000
                       ...   
          5          0.000652
          5         -0.000631
          5          0.000631
          5         -0.000631
          5          0.000631
Name: wap, Length: 302, dtype: float64

In [26] used 0.0000 MiB RAM in 0.13s, peaked 0.00 MiB above current, total RAM usage 428.05 MiB


In [58]:
from numba import njit

@njit(parallel=True)
def log_return_v2_inner(arr):
    arr = np.log(arr)
    arr2 = np.concatenate((np.array([np.NaN]), arr[1:] - arr[:-1]))
    return arr2

def log_return_v2(list_stock_prices_ser):
    # assume list_stock_prices is an array, not Series
    #print(type(list_stock_prices))
    list_stock_prices = list_stock_prices_ser.to_numpy()
    arr2 = log_return_v2_inner(list_stock_prices)
    return pd.Series(arr2, index=list_stock_prices_ser.index)


def make_realized_volatility_v2(df_book_data, log_return_name='log_return', wap_colname='wap', weights=None):
    """Consume wap column"""
    logged_diff = df_book_data.groupby(['stock_id', 'time_id'])[wap_colname].apply(log_return_v2)
    #print(logged_diff[:5])
    #df_book_data = df_book_data[~df_book_data[log_return_name].isnull()]
    logged_diff = logged_diff.dropna()
    df_realized_vol_per_stock =  pd.DataFrame(logged_diff.groupby(['stock_id', 'time_id']).agg(realized_volatility_weighted, weights))
    return df_realized_vol_per_stock

make_wap(df_book_train_stock_X, 1) # adds 'wap' column

df_realized_vol_per_stockX_orig = make_realized_volatility_v2(df_book_train_stock_X, log_return_name="log_return2", weights='linear')
df_realized_vol_per_stockX = make_realized_volatility_v2(df_book_train_stock_X, log_return_name="log_return2", weights='linear')
display(df_realized_vol_per_stockX[:5])
pd.testing.assert_frame_equal(df_realized_vol_per_stockX_orig, df_realized_vol_per_stockX)
#log_return_v2(ser.to_numpy()) # this looks like the original, sans the index

wap
stock_id time_id          
0        5        0.002517
         11       0.000904
         16       0.001504
         31       0.001665
         62       0.001402

In [58] used 1.0078 MiB RAM in 4.53s, peaked 1.62 MiB above current, total RAM usage 633.25 MiB


In [76]:
log_return_v2_inner.parallel_diagnostics(level=1)

 
 Parallel Accelerator Optimizing:  Function log_return_v2_inner, 
/tmp/ipykernel_66873/1514267201.py (3)  


Parallel loop listing for  Function log_return_v2_inner, /tmp/ipykernel_66873/1514267201.py (3) 
-----------------------------------------------------------------------|loop #ID
@njit(parallel=True)                                                   | 
def log_return_v2_inner(arr):                                          | 
    arr = np.log(arr)--------------------------------------------------| #0
    arr2 = np.concatenate((np.array([np.NaN]), arr[1:] - arr[:-1]))----| #1
    return arr2                                                        | 
------------------------------ After Optimisation ------------------------------
Parallel structure is already optimal.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
 
In [76] used 0.0000 MiB RAM in 0.11s, peaked 0.00 MiB

In [77]:
%timeit make_realized_volatility_v2(df_book_train_stock_X, log_return_name="log_return2", weights='linear')

1.63 s ± 46.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
In [77] used -0.0117 MiB RAM in 13.15s, peaked 1.32 MiB above current, total RAM usage 634.09 MiB


In [57]:
%timeit make_realized_volatility(df_book_train_stock_X, log_return_name="log_return2", weights='linear')

2.11 s ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
In [57] used 16.2188 MiB RAM in 16.99s, peaked 2.71 MiB above current, total RAM usage 632.24 MiB


In [63]:
%lprun -f make_realized_volatility_v2 -f realized_volatility_weighted -f _realized_volatility_weighted_sub -f log_return_v2 make_realized_volatility_v2(df_book_train_stock_X, log_return_name="log_return2", weights='linear')

In [63] used 0.5625 MiB RAM in 4.13s, peaked 0.54 MiB above current, total RAM usage 633.83 MiB


Timer unit: 1e-06 s

Total time: 1.08014 s
File: /tmp/ipykernel_66873/1514267201.py
Function: log_return_v2 at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def log_return_v2(list_stock_prices_ser):
    10                                               # assume list_stock_prices is an array, not Series
    11                                               #print(type(list_stock_prices))
    12      3830      58373.0     15.2      5.4      list_stock_prices = list_stock_prices_ser.to_numpy()
    13      3830     403553.0    105.4     37.4      arr2 = log_return_v2_inner(list_stock_prices)
    14      3830     618214.0    161.4     57.2      return pd.Series(arr2, index=list_stock_prices_ser.index)

Total time: 4.02051 s
File: /tmp/ipykernel_66873/1514267201.py
Function: make_realized_volatility_v2 at line 17

Line #      Hits         Time  Per Hit   % Time  Line Contents
    17                                       

In [17]:
#@memory.cache
def load_data_build_features(stock_id, ROOT, book_filename, trade_filename, cols, df_target):
    # filename e.g. book_train.parquet
    assert isinstance(stock_id, int)
    df_book_stock_X = pd.read_parquet(
        os.path.join(ROOT, f"{book_filename}/stock_id={stock_id}")
    )
    df_book_stock_X["stock_id"] = stock_id
    df_book_stock_X = df_book_stock_X.set_index(['stock_id', 'time_id'])
    #assert df_book_train_stock_X.shape[0] > rows_for_stock_id_0, (df_book_train_stock_X.shape[0], rows_for_stock_id_0)
    
    df_trade_stock_X = pd.read_parquet(
        os.path.join(ROOT, f"{trade_filename}/stock_id={stock_id}")
    )
    df_trade_stock_X["stock_id"] = stock_id
    df_trade_stock_X = df_trade_stock_X.set_index(['stock_id', 'time_id'])
    
    #df_book_train_stock_X_gt500 = df_book_train_stock_X.query("seconds_in_bucket>500").copy()
    #df_realized_vol_per_stock_short500 = add_wap_make_realized_volatility(df_book_train_stock_X_gt500, log_return_name='log_return_gt500sec')
    #df_book_train_stock_X_gt300 = df_book_train_stock_X.query("seconds_in_bucket>300").copy()
    #df_realized_vol_per_stock_short300 = add_wap_make_realized_volatility(df_book_train_stock_X_gt300, log_return_name='log_return_gt300sec')
    if True:
        make_wap(df_book_stock_X, 2, "wap2") 
        df_realized_vol_per_stock_wap2_uniform = make_realized_volatility(df_book_stock_X, log_return_name="log_return2_uniform", wap_colname="wap2", weights='uniform')    
        df_realized_vol_per_stock_wap2_linear = make_realized_volatility(df_book_stock_X, log_return_name="log_return2_linear", wap_colname="wap2", weights='linear')
        df_realized_vol_per_stock_wap2_half0half1 = make_realized_volatility(df_book_stock_X, log_return_name="log_return2_half0half1", wap_colname="wap2", weights='half0half1')
        make_wap(df_book_stock_X, 1, "wap") # adds 'wap' column
        df_realized_vol_per_stock_wap1_uniform = make_realized_volatility(df_book_stock_X, log_return_name="log_return1_uniform", weights='uniform')
        df_realized_vol_per_stock_wap1_linear = make_realized_volatility(df_book_stock_X, log_return_name="log_return1_linear", weights='linear')
        df_realized_vol_per_stock_wap1_half0half1 = make_realized_volatility(df_book_stock_X, log_return_name="log_return1_half0half1", weights='half0half1')

        features_var1 = make_features_stats(df_book_stock_X, 'var', cols)
        features_mean1 = make_features_stats(df_book_stock_X, 'mean', cols)
        features_size1 = make_features_stats(df_book_stock_X, 'size', cols)
        features_min1 = make_features_stats(df_book_stock_X, 'min', cols)
        features_max1 = make_features_stats(df_book_stock_X, 'max', cols)
        features_nunique1 = make_features_stats(df_book_stock_X, 'nunique', cols)

        df_book_stock_X['ask1_bid1_diff'] = (df_book_stock_X['ask_price1'] / df_book_stock_X['bid_price1']) - 1
        df_realized_vol_ask1_bid1_diff_per_stock = make_volatility_ask_bid_diff(df_book_stock_X, col='ask1_bid1_diff')
        df_realized_vol_ask1_bid1_diff_per_stock_linear = make_volatility_ask_bid_diff(df_book_stock_X, col='ask1_bid1_diff', weights='linear')
        df_realized_vol_ask1_bid1_diff_per_stock_linear = make_volatility_ask_bid_diff(df_book_stock_X, col='ask1_bid1_diff', weights='half0half1')
        df_book_stock_X['ask2_bid2_diff'] = (df_book_stock_X['ask_price2'] / df_book_stock_X['bid_price2']) - 1
        df_realized_vol_ask2_bid2_diff_per_stock = make_volatility_ask_bid_diff(df_book_stock_X, col='ask2_bid2_diff')
        df_realized_vol_ask2_bid2_diff_per_stock = make_volatility_ask_bid_diff(df_book_stock_X, col='ask2_bid2_diff', weights='linear')
        df_realized_vol_ask2_bid2_diff_per_stock = make_volatility_ask_bid_diff(df_book_stock_X, col='ask2_bid2_diff', weights='half0half1')
        df_book_stock_X['ask1_bid2_diff'] = (df_book_stock_X['ask_price1'] / df_book_stock_X['bid_price2']) - 1
        df_realized_vol_ask1_bid2_diff_per_stock = make_volatility_ask_bid_diff(df_book_stock_X, col='ask1_bid2_diff')
        df_book_stock_X['ask2_bid1_diff'] = (df_book_stock_X['ask_price2'] / df_book_stock_X['bid_price1']) - 1
        df_realized_vol_ask2_bid1_diff_per_stock = make_volatility_ask_bid_diff(df_book_stock_X, col='ask2_bid1_diff')
    else:
        features_var1 = make_features_stats(df_book_stock_X, 'var', cols)
        
    #breakpoint()
    # trade stats
    df_trade_basic_stats = df_trade_stock_X.groupby(['stock_id', 'time_id']).agg(trade_size_count=pd.NamedAgg('size', 'count'), trade_order_count_sum=pd.NamedAgg('order_count', 'sum'))

    df_train_stock_X = df_target.query('stock_id == @stock_id')
    if True:
        to_merge_book = [df_train_stock_X, 
                    features_var1, features_mean1, features_size1, 
                    features_min1, features_max1, features_nunique1,
                    df_realized_vol_per_stock_wap1_uniform,
                    df_realized_vol_ask1_bid1_diff_per_stock_linear,
                    df_realized_vol_per_stock_wap2_uniform,
                    df_realized_vol_per_stock_wap1_linear,
                    df_realized_vol_per_stock_wap2_linear,
                    df_realized_vol_per_stock_wap1_half0half1,
                    df_realized_vol_per_stock_wap2_half0half1,
                    df_realized_vol_ask1_bid1_diff_per_stock,
                    df_realized_vol_ask2_bid2_diff_per_stock,
                    df_realized_vol_ask1_bid2_diff_per_stock,
                    df_realized_vol_ask2_bid1_diff_per_stock,]
        to_merge_trade = [df_trade_basic_stats]
    else:
        to_merge_book = [df_train_stock_X, features_var1]
        to_merge_trade = [df_trade_basic_stats]

    # some trade datasets are missing some time_ids, making the join a mess
    # we reindex and make the choice to fillna 0
    to_merge_trade = [to_merge_tr.reindex(to_merge_book[0].index, fill_value=0) for to_merge_tr in to_merge_trade]
    #to_merge_trade = [to_merge_tr.reindex(to_merge_book[0].index, fill_value=0).fillna(0) for to_merge_tr in to_merge_trade]
    
    row_lengths = [df.shape[0] for df in to_merge_book]
    assert len(set(row_lengths)) == 1, f"row_lengths are different for stock {stock_id}: {row_lengths}" # should all be same length
    to_merge = to_merge_book + to_merge_trade
    for idx, item_to_merge in enumerate(to_merge):
        assert item_to_merge.index.names == ['stock_id', 'time_id'], f"We must have the same index on idx {idx}"
    train_merged = pd.concat(to_merge, axis=1)
                             
    if 'target' in train_merged.columns:
        features = train_merged.drop(columns='target').columns
        #print(features)
        assert len(set(features)) == len(features), f"Feature duplication! {len(set(features))} vs {len(features)}"

    return train_merged

#if 'memory' in dir():
#    # only setup local cache if we're running locally in development
#    load_data_build_features = memory.cache(load_data_build_features)
    
cols = ['bid_price1', 'ask_price1', 'bid_price2', 'ask_price2',] 
cols += ['bid_size1', 'ask_size1', 'bid_size2', 'ask_size2']

# test...
train_mergedXX = load_data_build_features(31, ROOT, 'book_train.parquet', 'trade_train.parquet', cols, df_train_all)
display(train_mergedXX.head(5))

%lprun -f load_data_build_features load_data_build_features(31, ROOT, 'book_train.parquet', 'trade_train.parquet', cols, df_train_all)

target  bid_price1_var  ask_price1_var  bid_price2_var  \
stock_id time_id                                                             
31       5        0.004113    2.692188e-06    2.681080e-06    2.692195e-06   
         11       0.000956    2.032095e-08    2.031816e-08    2.032095e-08   
         16       0.002127    1.158020e-06    1.158066e-06    1.157965e-06   
         31       0.003748    4.796240e-07    4.796240e-07    4.796240e-07   
         62       0.001573    2.165609e-07    2.166966e-07    2.165884e-07   

                  ask_price2_var  bid_size1_var  ask_size1_var  bid_size2_var  \
stock_id time_id                                                                
31       5          2.681076e-06   2.238556e+07   3.623230e+07   1.806015e+07   
         11         2.031816e-08   4.570735e+07   1.831557e+08   1.766419e+07   
         16         1.157958e-06   1.800935e+08   2.033910e+08   6.437634e+08   
         31         4.796240e-07   8.473959e+07   4.096441e+07   3.710387e+07   
         62         2.167498e-07   1.690191e+08   1.316160e+08   6.440620e+07   

                  ask_size2_var  bid_price1_mean  ...  log_return1_linear  \
stock_id time_id                                  ...                       
31       5         1.258087e+07         0.997129  ...            0.002836   
         11        1.628047e+08         1.000411  ...            0.000324   
         16        2.730231e+08         0.999483  ...            0.001487   
         31        1.243901e+08         0.994919  ...            0.001739   
         62        7.843908e+07         0.999969  ...            0.001121   

                  log_return2_linear  log_return1_half0half1  \
stock_id time_id                                               
31       5                  0.003074                0.003343   
         11                 0.001223                0.000292   
         16                 0.002725                0.001602   
         31                 0.004240                0.002036   
         62                 0.001908                0.000994   

                  log_return2_half0half1  ask1_bid1_diff_log_return_uniform  \
stock_id time_id                                                              
31       5                      0.003636                           0.981440   
         11                     0.000252                           0.001358   
         16                     0.002811                           0.002366   
         31                     0.005161                           0.003457   
         62                     0.001528                           0.980168   

                  ask2_bid2_diff_log_return_half0half1  \
stock_id time_id                                         
31       5                                    0.408059   
         11                                   0.000000   
         16                                   0.001364   
         31                                   0.002758   
         62                                   0.406906   

                  ask1_bid2_diff_log_return_uniform  \
stock_id time_id                                      
31       5                                 0.574537   
         11                                0.001260   
         16                                0.002336   
         31                                0.003121   
         62                                0.573358   

                  ask2_bid1_diff_log_return_uniform  trade_size_count  \
stock_id time_id                                                        
31       5                                 0.574467                50   
         11                                0.001261                10   
         16                                0.002442                12   
         31                                0.003269                30   
         62                                0.573510                21   

                  trade_order_count_sum  
stock_id time_i

In [17] used 33.4922 MiB RAM in 118.91s, peaked 524.11 MiB above current, total RAM usage 414.73 MiB


Timer unit: 1e-06 s

Total time: 77.6965 s
File: /tmp/ipykernel_66873/831036116.py
Function: load_data_build_features at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def load_data_build_features(stock_id, ROOT, book_filename, trade_filename, cols, df_target):
     3                                               # filename e.g. book_train.parquet
     4         1         14.0     14.0      0.0      assert isinstance(stock_id, int)
     5         2     244754.0 122377.0      0.3      df_book_stock_X = pd.read_parquet(
     6         1         39.0     39.0      0.0          os.path.join(ROOT, f"{book_filename}/stock_id={stock_id}")
     7                                               )
     8         1       3274.0   3274.0      0.0      df_book_stock_X["stock_id"] = stock_id
     9         1     117988.0 117988.0      0.2      df_book_stock_X = df_book_stock_X.set_index(['stock_id', 'time_id'])
    10            